In [1]:
# import libraries
import pandas as pd
import numpy as np


In [2]:
pwd

'/Users/austinbrian/dev/sheriffs/notebooks'

In [3]:
ct = pd.read_csv('../data/ct.csv',dtype={'FIPS':object})
pres = pd.read_csv('../data/president_counties.csv')
pres['turnout'] =  pres.total_votes/pres.people
cols = [i for i in pres.columns]
cols.append('fips_county')
pres['fips_countyname_county'] = [str(i.split(',')[0])+' County' 
                                  for i in pres.countyname]
pres['state'] = [str(i.split(',')[1][1:]) for i in pres.countyname]

# pres = pd.merge(left=pres,right=ct,how='left',left_on='FIPS',right_on='FIPS')

In [4]:
ct[ct.FIPS=='10003']

,FIPS,fips_county,countyname,Reporting,trump,clinton,johnson,Others,stein,Darrell Castle,...,Emidio Soltysik,Scott Copeland,Jerry White,Monica Moorehead,Richard Duncan,Peter Skewes,Rocky Giordani,total_votes,state,ind_cand
290,10003,"New Castle County, DE","New Castle, DE",100.0,85525,162919,9107,1472,3956,0,...,0,0,0,0,0,0,0,262979,DE,14535


In [5]:
pres.shape

(3113, 17)

In [6]:
pres.head()

,FIPS,countyname,state,clinton,trump,johnson,ind_cand,mcmullin,total_votes,AGI,num_returns,num_dependents,num_tot_inc,tot_inc_amt,people,turnout,fips_countyname_county
0,2000,"Alaska, AK",AK,93007,130415,14593,23166,0,246588,24270413.0,361110.0,209280.0,360720.0,24628251.0,570390.0,0.432315,Alaska County
1,1107,"Pickens, AL",AL,3972,5456,49,114,0,9542,301640.0,7530.0,6000.0,7500.0,305093.0,13530.0,0.705248,Pickens County
2,1007,"Bibb, AL",AL,1874,6738,124,207,0,8819,365681.0,7980.0,6340.0,7960.0,369326.0,14320.0,0.615852,Bibb County
3,1079,"Lawrence, AL",AL,3627,10833,250,369,0,14829,601382.0,13540.0,9010.0,13510.0,606944.0,22550.0,0.657605,Lawrence County
4,1115,"St. Clair, AL",AL,5589,31651,684,1160,0,38400,1794459.0,35230.0,24400.0,35140.0,1814483.0,59630.0,0.643971,St. Clair County


### Detention Data
Need to build a scraper for detention - the point is to go through each detention center in each state and record the yes/no ratio for each. 

In [7]:
import requests
from bs4 import BeautifulSoup
import selenium

ModuleNotFoundError: No module named 'selenium'

## ILRC 
This stuff is embedded in a google map [on their blog](https://www.ilrc.org/local-enforcement-map). Probably best just to ask for the raw file. But it's possible that the data is scrapable. The class that looks for the relevant information is below.

###### relevant html: 
```html
<div class="googft-info-window">
```

Source for [fusion table](https://fusiontables.googleusercontent.com/embedviz?q=select+col13+from+1x4mRJ-IKLOWgSAYGHWULvshcVoGHGiXftK3kZb4_&viz=MAP&h=false&lat=37.98471683444062&lng=-92.96410977109372&t=1&z=5&l=col13&y=2&tmplt=2&hml=KML): `1x4mRJ-IKLOWgSAYGHWULvshcVoGHGiXftK3kZb4_`

In [8]:
ilrc = pd.read_csv('../data/County_Imm_Map_ILRC.csv',dtype={"GEOID - FIPS":object})
ilrc.head().T

,0,1,2,3,4
County,State of Rhode Island,Autauga,Baldwin,Barbour,Bibb
Field Office,BOS,NOL,NOL,NOL,NOL
State,RI,AL,AL,AL,AL
Jurisdiction,Rhode Island Department of Corrections (ACI),Autauga Sheriffs Office,Baldwin Sheriffs Office,Barbour Sheriffs Office,Bibb Sheriffs Office
Priority,C: Large jurisdictions that continue to accept...,D: Remaining jurisdictions that continue to ac...,C: Large jurisdictions that continue to accept...,D: Remaining jurisdictions that continue to ac...,D: Remaining jurisdictions that continue to ac...
Month and Year Jurisdiction Began Not Accepting Detainers,NaN,NaN,NaN,NaN,NaN
Date of First Engagement between ERO and Local Government,6/22/2015,7/31/2015,7/31/2015,7/31/2015,7/31/2015
Date of Last Engagement between ERO and Local Government,9/1/2015,7/31/2015,7/31/2015,7/31/2015,7/31/2015
Who Met with the Local Government?,AFOD,FOD,FOD,FOD,FOD
Current Detainer/Notification Acceptance Status,"2: Considering, but (currently) not willing to...",4: Willing to accept both (I-247N) Notificatio...,4: Willing to accept both (I-247N) Notificatio...,4: Willing to accept both (I-247N) Notificatio...,4: Willing to accept both (I-247N) Notificatio...


In [9]:
ilrc.columns

Index(['County', 'Field Office', 'State', 'Jurisdiction', 'Priority',
       'Month and Year Jurisdiction Began Not Accepting Detainers',
       'Date of First Engagement between ERO and Local Government',
       'Date of Last Engagement between ERO and Local Government',
       'Who Met with the Local Government?',
       'Current Detainer/Notification Acceptance Status', 'ICE Comments',
       'Follow Up Meeting Scheduled', 'Next Meeting Date',
       'HQ Priority Jurisdictions (187)', 'ILRC Analysis', 'No 287(g)',
       'No ICE Detention Contract', 'No ICE holds', 'No ICE alerts',
       'Limits on ICE interrogations in jail',
       'Prohibition on asking about immigration status',
       'General prohibition on assistance to ICE', 'ILRC Total', 'Path',
       'ILRC Comments', 'Title of Policy', 'LINK', 'GEOID - FIPS'],
      dtype='object')

### Immigration Data
From the Migration Policy Institute

In [11]:
google_doc_data = 'https://docs.google.com/spreadsheets/d/1Iq0zcvQUskwYWCieayvIF8kaFGONxa9ZkCELyQzbbZI/edit#gid=816492683'


In [10]:
imm = pd.read_csv('../data/State-county-unauthorized-estimates.csv')
imm['county_split']=[i.split(',') for i in imm.County]
imm['county_name']=[i[0] for i in imm.county_split]
imm = imm.drop('county_split',axis=1)
imm['countyname_split']=[i.split(' ') for i in imm.county_name]
imm['County_UA_pop'] = [int(i.replace(',','')) for i in imm['Total Unauthorized Population']]
imm.drop('Total Unauthorized Population',axis=1, inplace=True)
# There's sometimes a couple of counties all together
imm['mult_counties'] = 0
for i,v in enumerate(imm.countyname_split):
    if v[-1] == 'Counties':
        imm.loc[i,'mult_counties'] = 1
imm = imm.drop('countyname_split',axis=1)
print(imm.shape)
imm.head()

(171, 5)


,State,County,county_name,County_UA_pop,mult_counties
0,California,"Los Angeles County, California",Los Angeles County,1060000,0
1,Texas,"Harris County, Texas",Harris County,376000,0
2,Illinois,"Cook County, Illinois",Cook County,325000,0
3,California,"Orange County, California",Orange County,270000,0
4,New York,"Queens County, New York",Queens County,232000,0


In [11]:
imm.State.nunique()

39

In [12]:

ua_state = pd.read_csv('../data/State-unauthorized-estimates.csv')
ua_state['Statewide_UA_pop'] = [int(i.replace(',','')) for i in ua_state['Total Unauthorized Population']]
ua_state.drop('Total Unauthorized Population',axis=1, inplace=True)
print(ua_state.shape)
ua_state.head()


(48, 3)


,State,State_init,Statewide_UA_pop
0,California,CA,3019000
1,Texas,TX,1470000
2,New York,NY,850000
3,Florida,FL,610000
4,Illinois,IL,519000


In [13]:

imm_ua = pd.merge(left=imm,right=ua_state,left_on='State',right_on='State')
pres=pd.merge(left=pres, right=imm_ua[['State','State_init']], left_on='state',right_on='State_init')
print(imm_ua.shape)
imm_ua.head()

(171, 7)


,State,County,county_name,County_UA_pop,mult_counties,State_init,Statewide_UA_pop
0,California,"Los Angeles County, California",Los Angeles County,1060000,0,CA,3019000
1,California,"Orange County, California",Orange County,270000,0,CA,3019000
2,California,"San Diego County, California",San Diego County,207000,0,CA,3019000
3,California,"Riverside County, California",Riverside County,161000,0,CA,3019000
4,California,"San Bernardino County, California",San Bernardino County,140000,0,CA,3019000


In [14]:
pres.head()

,FIPS,countyname,state,clinton,trump,johnson,ind_cand,mcmullin,total_votes,AGI,num_returns,num_dependents,num_tot_inc,tot_inc_amt,people,turnout,fips_countyname_county,State,State_init
0,1107,"Pickens, AL",AL,3972,5456,49,114,0,9542,301640.0,7530.0,6000.0,7500.0,305093.0,13530.0,0.705248,Pickens County,Alabama,AL
1,1007,"Bibb, AL",AL,1874,6738,124,207,0,8819,365681.0,7980.0,6340.0,7960.0,369326.0,14320.0,0.615852,Bibb County,Alabama,AL
2,1079,"Lawrence, AL",AL,3627,10833,250,369,0,14829,601382.0,13540.0,9010.0,13510.0,606944.0,22550.0,0.657605,Lawrence County,Alabama,AL
3,1115,"St. Clair, AL",AL,5589,31651,684,1160,0,38400,1794459.0,35230.0,24400.0,35140.0,1814483.0,59630.0,0.643971,St. Clair County,Alabama,AL
4,1109,"Pike, AL",AL,5056,7693,257,419,0,13168,552172.0,12620.0,8740.0,12580.0,560651.0,21360.0,0.616479,Pike County,Alabama,AL


In [15]:
imm_ua.State.nunique()

39

In [16]:
imm_ua.shape

(171, 7)

In [18]:
pres.state.nunique()

39

In [19]:
df.FIPS.nunique()

NameError: name 'df' is not defined

In [20]:
df =pd.merge(left=pres,right=imm_ua,left_on=['fips_countyname_county','state'],
             right_on=['county_name','State_init'])
print(df.shape)
df.head().T

(1696, 26)


,0,1,2,3,4
FIPS,1073,5007,5007,5143,5143
countyname,"Jefferson, AL","Benton, AR","Benton, AR","Washington, AR","Washington, AR"
state,AL,AR,AR,AR,AR
clinton,156873,28005,28005,33366,33366
trump,134768,60871,60871,41476,41476
johnson,6645,3871,3871,3360,3360
ind_cand,12550,7948,7948,7019,7019
mcmullin,0,2313,2313,1438,1438
total_votes,304191,96824,96824,81861,81861
AGI,1.87056e+07,9.64608e+06,9.64608e+06,5.08819e+06,5.08819e+06


In [21]:
merged_counties = [i for i in df.fips_countyname_county]
print("{} merged".format(len(merged_counties)))
immigration_counties = [i for i in imm.county_name]
to_be_merged = [i for i in immigration_counties if i not in merged_counties]
print("{} to be merged".format(len(to_be_merged)))

1696 merged
26 to be merged


In [30]:
# Checking the number of counties that did/didn't merge; mostly because of multi-counties or NECTA areas
for i in immigration_counties:
    if i not in merged_counties:
        print(i)

Miami-Dade-Monroe Counties
Boston Area NECTA*
Prince Georges County
New York County
Monterey-San Benito Counties
Suburban Denver Counties**
Fairfield County
De Kalb County
Du Page County
Montgomery-Chambers-Liberty Counties
Providence County
New Haven County
Hartford County
Jefferson-Plaquemines-St. Bernard Parishes
Baltimore City
Richmond County
Jackson-Cass Counties
Alexandria City
Orange-Chatham Counties
Duval-Nassau Counties
Sedgwick-Butler-Harvey Counties
Worcester Area NECTA*
St. Louis County
Beaufort-Jasper Counties
Doña Ana County
Knox-Anderson-Union Counties


In [184]:
# df.to_csv('./data/immigration.csv',index=False)

In [185]:
# df = pd.read_csv('./data/immigration.csv')

In [43]:
imm_df = pd.read_csv('../data/immigration.csv')
ice = pd.read_csv('../data/county_rollup_ice_detainees.csv')
ice['pct_ice']=ice.Yes/ice.All
ice.rename(columns={'All':'total_detained','Yes':'ICE_custody','No':'No_ICE_custody'},inplace=True)
imm = pd.merge(left=imm,right=ice,how='left',left_on=['county_name','State'],
             right_on=['County','State'])

In [44]:
imm_df.head()

,FIPS,countyname,state,clinton,trump,johnson,ind_cand,mcmullin,total_votes,AGI,...,people,turnout,fips_countyname_county,State,County,county_name,County_UA_pop,mult_counties,State_init,Statewide_UA_pop
0,1073,"Jefferson, AL",AL,156873,134768,6645,12550,0,304191,18705559.0,...,510200.0,0.596219,Jefferson County,Alabama,"Jefferson County, Alabama",Jefferson County,13000,0,AL,71000
1,5007,"Benton, AR",AR,28005,60871,3871,7948,2313,96824,9646077.0,...,186370.0,0.519526,Benton County,Arkansas,"Benton County, Arkansas",Benton County,11000,0,AR,56000
2,5143,"Washington, AR",AR,33366,41476,3360,7019,1438,81861,5088192.0,...,159750.0,0.512432,Washington County,Arkansas,"Washington County, Arkansas",Washington County,11000,0,AR,56000
3,4027,"Yuma, AZ",AZ,24605,25165,1873,3240,0,53010,3258575.0,...,165690.0,0.319935,Yuma County,Arizona,"Yuma County, Arizona",Yuma County,13000,0,AZ,244000
4,4021,"Pinal, AZ",AZ,47892,72819,5010,8835,0,129546,6370328.0,...,243290.0,0.532476,Pinal County,Arizona,"Pinal County, Arizona",Pinal County,11000,0,AZ,244000


In [46]:
imm_df.countyname.nunique()

144

In [47]:
ice.head()

,State,County,total_detained,ICE_custody,No_ICE_custody,pct_ice
0,Alabama,All,9943,4482.0,5461.0,0.450769
1,Alabama,Autauga County,67,45.0,22.0,0.671642
2,Alabama,Baldwin County,738,401.0,337.0,0.543360
3,Alabama,Barbour County,11,5.0,6.0,0.454545
4,Alabama,Bibb County,14,13.0,1.0,0.928571


In [10]:
ice[ice.State=='Florida'].sort_values(by='pct_ice',ascending=False).head(20)

,State,County,total_detained,ICE_custody,No_ICE_custody,pct_ice
306,Florida,Franklin County,10,9.0,1.0,0.900000
299,Florida,Collier County,6356,5259.0,1097.0,0.827407
302,Florida,Dixie County,25,20.0,5.0,0.800000
315,Florida,Highlands County,483,372.0,111.0,0.770186
303,Florida,Duval County,2511,1920.0,591.0,0.764636
349,Florida,Suwannee County,186,135.0,51.0,0.725806
330,Florida,Martin County,1427,1033.0,394.0,0.723896
318,Florida,Indian River County,727,510.0,217.0,0.701513
307,Florida,Gadsden County,144,101.0,43.0,0.701389
335,Florida,Okeechobee County,429,297.0,132.0,0.692308


In [11]:
imm['taxpayer_turnout'] = imm.total_votes/imm.num_returns
imm['pct_clinton'] = imm.clinton/imm.total_votes
imm['pct_ua_pop_detained'] = imm.total_detained/imm.County_UA_pop
imm['log_total_detained'] = np.log(imm.total_detained)
imm.head().T

,0,1,2,3,4
FIPS,1073,5007,5143,4027,4021
countyname,"Jefferson, AL","Benton, AR","Washington, AR","Yuma, AZ","Pinal, AZ"
state,AL,AR,AR,AZ,AZ
clinton,156873,28005,33366,24605,47892
trump,134768,60871,41476,25165,72819
johnson,6645,3871,3360,1873,5010
ind_cand,12550,7948,7019,3240,8835
mcmullin,0,2313,1438,0,0
total_votes,304191,96824,81861,53010,129546
AGI,1.87056e+07,9.64608e+06,5.08819e+06,3.25858e+06,6.37033e+06


In [189]:
imm.shape

(144, 33)

In [190]:
imm[imm.total_detained>1000].shape

(126, 33)

## Plotting

In [191]:
df.head()

,FIPS,countyname,state,clinton,trump,johnson,ind_cand,mcmullin,total_votes,AGI,...,fips_countyname_county,State_x,State_init_x,State_y,County,county_name,County_UA_pop,mult_counties,State_init_y,Statewide_UA_pop
0,1073,"Jefferson, AL",AL,156873,134768,6645,12550,0,304191,18705559.0,...,Jefferson County,Alabama,AL,Alabama,"Jefferson County, Alabama",Jefferson County,13000,0,AL,71000
1,5007,"Benton, AR",AR,28005,60871,3871,7948,2313,96824,9646077.0,...,Benton County,Arkansas,AR,Arkansas,"Benton County, Arkansas",Benton County,11000,0,AR,56000
2,5007,"Benton, AR",AR,28005,60871,3871,7948,2313,96824,9646077.0,...,Benton County,Arkansas,AR,Arkansas,"Benton County, Arkansas",Benton County,11000,0,AR,56000
3,5143,"Washington, AR",AR,33366,41476,3360,7019,1438,81861,5088192.0,...,Washington County,Arkansas,AR,Arkansas,"Washington County, Arkansas",Washington County,11000,0,AR,56000
4,5143,"Washington, AR",AR,33366,41476,3360,7019,1438,81861,5088192.0,...,Washington County,Arkansas,AR,Arkansas,"Washington County, Arkansas",Washington County,11000,0,AR,56000


In [12]:
import matplotlib.pyplot as plt, mpld3
detained_num = 10
df = imm[imm.total_detained>detained_num]
df = df[df.pct_ice<100]
x = df.pct_clinton
x = df.total_detained
y = df.pct_ice
area = 250
text = df['countyname']
colors = df.pct_clinton
cm = plt.cm.get_cmap('RdBu')


plt.rc('font', weight='bold', family='sans-serif', size=12)


fig,ax = plt.subplots(figsize=(12,12))
sct = ax.scatter(x, y, s=area, linewidths=2, edgecolor='w')
# sct = plt.scatter(x, y, linewidths=10, edgecolor='w')
# plugins.connect(fig, plugins.PointLabelTooltip(fig))
sct.set_alpha(0.75)

for label, X, Y in zip(text, x, y):
    ax.annotate(label,xy=(x, y))

hfont = {'fontname':'DejaVu Sans'}

ax.set_xlabel('Total Number Detained', fontsize=16,**hfont)
ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
ax.set_title('Counties with More than {} Undocumented'.format(detained_num),fontsize=22,**hfont)


# sct_html = fig_to_html(fig)
# mpld3.save_html(fig,'../findings/nationwide_10k.html')
mpld3.display()

TypeError: cannot convert the series to <type 'float'>

In [43]:
import matplotlib.pyplot as plt, mpld3
detained_num = 1000
df = imm[imm.total_detained>detained_num]
x = df.total_detained
y = df.pct_ice
colors = df.pct_clinton
cm = plt.cm.get_cmap('RdBu')
# area = df.total_detained/50
text = df['County_x']

plt.rc('font', weight='bold', family='sans-serif', size=12)


fig,ax = plt.subplots(figsize=(10,10))
sct = ax.scatter(x, y, s=area, linewidths=2, edgecolor='w',c=colors, cmap=cm)
# sct = plt.scatter(x, y, linewidths=10, edgecolor='w')
# plugins.connect(fig, plugins.PointLabelTooltip(fig))
sct.set_alpha(0.75)

for label, x, y in zip(text, x, y):
    ax.annotate(label,xy=(x, y))

hfont = {'fontname':'DejaVu Sans'}

ax.set_xlabel('Number of Detentions', fontsize=16,**hfont)
ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
ax.set_title('Nationwide Undocumented'.format(detained_num),fontsize=22,**hfont)


# sct_html = fig_to_html(fig)
mpld3.save_html(fig,'../findings/above_10k_RdBu.html')
mpld3.display()

ValueError: could not convert string to float: 

In [88]:
detained_num = 1000
df = imm[imm.total_detained>detained_num]
x = df.pct_clinton
y = df.pct_ice
area = df.total_detained
text = df['County_x']

plt.rc('font', weight='bold', family='sans-serif', size=12)


fig,ax = plt.subplots(figsize=(10,10))
sct = ax.scatter(x, y, s=area, linewidths=2, edgecolor='w')
# plugins.connect(fig, plugins.PointLabelTooltip(fig))
sct.set_alpha(0.75)

for labeli, xi, yi,si in zip(text, x, y,area):
    if si>detained_num:
        ax.annotate(labeli,xy=(xi, yi))

hfont = {'fontname':'DejaVu Sans'}

ax.set_xlabel('Percentage of County Vote for Clinton', fontsize=16,**hfont)
ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
ax.set_title('Counties with More than {} Detained'.format(detained_num),fontsize=22,**hfont)


# sct_html = fig_to_html(fig)
# mpld3.save_html(fig,'../findings/above_1000.html')
mpld3.display()

AttributeError: 'int' object has no attribute 'sqrt'

In [13]:
pres = pres.drop_duplicates('FIPS')
print pres.shape

(2692, 19)


In [14]:
statecountyimm = pd.merge(left=pres[['State','State_init','countyname',
                                     'fips_countyname_county','clinton','trump','total_votes']],
                          right=ice,
                          left_on=['fips_countyname_county','State'],
                          right_on=['County','State'])
statecountyimm['pct_clinton'] = statecountyimm.clinton/statecountyimm.total_votes
print statecountyimm.shape
statecountyimm['dem_votes_per_custody'] = statecountyimm.clinton/statecountyimm.ICE_custody
statecountyimm['dem_excess_per_custody'] = (statecountyimm.clinton-statecountyimm.trump)/statecountyimm.ICE_custody
statecountyimm = statecountyimm.drop_duplicates()
print statecountyimm.shape

(1983, 13)
(1983, 15)


In [15]:
statecountyimm[statecountyimm.total_detained>100].sort_values(by='dem_excess_per_custody',ascending=False).head(20)

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
1188,New York,NY,"Bronx, NY",Bronx County,353646,37797,399522,Bronx County,1261,308.0,953.0,0.244251,0.885173,1148.201299,1025.483766
1412,Pennsylvania,PA,"Allegheny, PA",Allegheny County,367617,259480,657189,Allegheny County,432,132.0,300.0,0.305556,0.559378,2784.977273,819.219697
830,Michigan,MI,"Wayne, MI",Wayne County,519444,228993,782719,Wayne County,1455,359.0,1096.0,0.246735,0.663640,1446.919220,809.055710
1212,New York,NY,"Monroe, NY",Monroe County,188592,136582,347790,Monroe County,242,67.0,175.0,0.276860,0.542258,2814.805970,776.268657
670,Indiana,IN,"Lake, IN",Lake County,116935,75625,202801,Lake County,137,56.0,81.0,0.408759,0.576600,2088.125000,737.678571
1280,Ohio,OH,"Lucas, OH",Lucas County,110833,75698,198830,Lucas County,124,54.0,70.0,0.435484,0.557426,2052.462963,650.648148
1259,Ohio,OH,"Cuyahoga, OH",Cuyahoga County,398271,184211,608873,Cuyahoga County,855,339.0,516.0,0.396491,0.654112,1174.840708,631.445428
46,Alabama,AL,"Jefferson, AL",Jefferson County,156873,134768,304191,Jefferson County,508,41.0,467.0,0.080709,0.515706,3826.170732,539.146341
1217,New York,NY,"Onondaga, NY",Onondaga County,112337,83649,208440,Onondaga County,158,57.0,101.0,0.360759,0.538942,1970.824561,503.298246
930,Missouri,MO,"Jackson, MO",Jackson County,168972,116211,304687,Jackson County,887,122.0,765.0,0.137542,0.554576,1385.016393,432.467213


In [16]:
statecountyimm.describe()

,clinton,trump,total_votes,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
count,1.983000e+03,1983.000000,1.983000e+03,1983.000000,1894.000000,1945.000000,1894.000000,1983.000000,1894.000000,1894.000000
mean,2.716073e+04,25812.649521,5.629990e+04,870.354009,528.739704,372.482776,0.495695,0.328036,465.043826,-463.545233
std,9.512173e+04,50855.594902,1.492439e+05,4404.435140,2918.624278,1721.948517,0.201080,0.145608,1066.899847,1499.683968
min,2.000000e+01,280.000000,4.230000e+02,1.000000,1.000000,1.000000,0.007752,0.036101,0.031590,-32362.000000
25%,1.816500e+03,4623.000000,7.124000e+03,17.000000,8.000000,11.000000,0.347826,0.221134,51.405263,-419.437500
50%,4.413000e+03,10108.000000,1.595200e+04,68.000000,33.000000,37.000000,0.500000,0.300523,135.740385,-121.372141
75%,1.417850e+04,24538.500000,4.214000e+04,288.500000,154.750000,142.000000,0.636364,0.408053,430.406250,-17.303503
max,2.464364e+06,769743.000000,3.434308e+06,85944.000000,62964.000000,37851.000000,1.000000,0.885173,23131.000000,3132.000000


In [29]:
df = statecountyimm
mask = (df.total_detained>10)&(df.pct_ice>.5)
logic = df.pct_clinton*df.pct_ice
# (df.clinton-df.trump)*df.pct_ice # keys uses total number of votes


df['test'] = logic
tosave = df[mask].sort_values(by='test', ascending=False).head(50)
tosave.to_csv('../findings/dem_perf_per_custody.csv')
# plt.countplot(x=a['logic'])

In [30]:
# Should get a list of sanctuary cities and check to see if each county has one in it

In [197]:
def graph(state, detained_num=0, save=False):
    df = statecountyimm[statecountyimm.State_init==state]
    df = df[df.total_detained>detained_num]
    x = df.total_detained
    y = df.pct_ice
    colors = df.pct_clinton
    cm = plt.cm.get_cmap('RdBu')
    area= 400
    # area = (df.total_votes/df.total_detained)*10
    text = df['countyname']

    plt.rc('font', weight='bold', family='sans-serif', size=12)


    fig,ax = plt.subplots(figsize=(10,10))
    sct = ax.scatter(x, y, linewidths=2, s=area, edgecolor='w',c=colors,cmap=cm)
    # plugins.connect(fig, plugins.PointLabelTooltip(fig))
    sct.set_alpha(0.75)

    # tooltip = mpld3.plugins.PointLabelTooltip(sct, labels=zip(labels,label2))
    # mpld3.plugins.connect(fig, tooltip)

    
    for labeli, xi, yi in zip(text, x, y):
        ax.annotate(labeli,xy=(xi, yi))
            
            
#     for labeli, xi, yi in zip(text, x, y):
#         if xi>(.1*max(x)):
#             ax.annotate(labeli,xy=(xi, yi))
#         if yi>(.9*max(y)):
#             ax.annotate(labeli,xy=(xi, yi))

    hfont = {'fontname':'DejaVu Sans'}

    
    ax.set_xlabel('Total Detained', fontsize=16,**hfont)
    ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
    ax.set_title('Counties in {}'.format(state),fontsize=22,**hfont)

    # sct_html = fig_to_html(fig)
    if save==True:
        mpld3.save_html(fig,'../findings/oct_23_max/{}_counties.html'.format(state))
    return mpld3.display()

graph('MD')

In [129]:
state_list = ['CA','MD','NC','IL','NV','CO']
for state in state_list:
    graph(state,save=True)

In [133]:
def graph_national(detained_num=0, save=False):
    df = statecountyimm
    df = df[df.total_detained>detained_num]
    x = df.total_detained
    y = df.pct_ice
    colors = df.pct_clinton
    cm = plt.cm.get_cmap('RdBu')
    area= 400
    # area = (df.total_votes/df.total_detained)*10
    text = df['countyname']

    plt.rc('font', weight='bold', family='sans-serif', size=12)


    fig,ax = plt.subplots(figsize=(10,10))
    sct = ax.scatter(x, y, linewidths=2, s=area, edgecolor='w',c=colors,cmap=cm)
    # plugins.connect(fig, plugins.PointLabelTooltip(fig))
    sct.set_alpha(0.75)

    # tooltip = mpld3.plugins.PointLabelTooltip(sct, labels=zip(labels,label2))
    # mpld3.plugins.connect(fig, tooltip)

    
#     for labeli, xi, yi in zip(text, x, y):
#         ax.annotate(labeli,xy=(xi, yi))
            
            
    for labeli, xi, yi in zip(text, x, y):
        if xi>(.1*max(x)):
            ax.annotate(labeli,xy=(xi, yi))
        if yi>(.9*max(y)):
            ax.annotate(labeli,xy=(xi, yi))

    hfont = {'fontname':'DejaVu Sans'}

    
    ax.set_xlabel('Total Detained', fontsize=16,**hfont)
    ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
    ax.set_title('Counties Nationwide'.format(state),fontsize=22,**hfont)

    # sct_html = fig_to_html(fig)
    if save==True:
        mpld3.save_html(fig,'../findings/oct_23_max/nationwide_counties.html'.format(state))
    return mpld3.display()

graph_national(detained_num=1000, save=True)

In [222]:
# Using document here to set up 2018 elections: https://www.sheriffs.org/sites/default/files/uploads/documents/GovAffairs/State-by-State%20Election%20Chart%20updated%2008.13.15.pdf
df = pd.DataFrame(columns = statecountyimm.columns)
statecountyimm.head()

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
0,Alabama,AL,"Pickens, AL",Pickens County,3972,5456,9542,Pickens County,331,220.0,111.0,0.664653,0.416265,18.054545,-6.745455
1,Alabama,AL,"Bibb, AL",Bibb County,1874,6738,8819,Bibb County,14,13.0,1.0,0.928571,0.212496,144.153846,-374.153846
2,Alabama,AL,"Lawrence, AL",Lawrence County,3627,10833,14829,Lawrence County,32,8.0,24.0,0.250000,0.244588,453.375000,-900.750000
3,Alabama,AL,"St. Clair, AL",St. Clair County,5589,31651,38400,St. Clair County,40,1.0,39.0,0.025000,0.145547,5589.000000,-26062.000000
4,Alabama,AL,"Pike, AL",Pike County,5056,7693,13168,Pike County,16,11.0,5.0,0.687500,0.383961,459.636364,-239.727273


In [164]:
len(statecountyimm)

1983

In [224]:
testdf = statecountyimm[statecountyimm.State_init == 'NY']
# testdf = testdf[testdf.County =='Chatham County']
testdf

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
6034,New York,NY,"Albany, NY",Albany County,83077,47810,139830,Albany County,404,180.0,224.0,0.445545,0.594129,461.538889,195.927778
6044,New York,NY,"Allegany, NY",Allegany County,4882,12525,18692,Allegany County,32,17.0,15.0,0.531250,0.261181,287.176471,-449.588235
6054,New York,NY,"Bronx, NY",Bronx County,353646,37797,399522,Bronx County,1261,308.0,953.0,0.244251,0.885173,1148.201299,1025.483766
6064,New York,NY,"Broome, NY",Broome County,39212,40943,86072,Broome County,83,46.0,37.0,0.554217,0.455572,852.434783,-37.630435
6074,New York,NY,"Cattaraugus, NY",Cattaraugus County,9497,19692,31161,Cattaraugus County,268,165.0,103.0,0.615672,0.304772,57.557576,-61.787879
6084,New York,NY,"Cayuga, NY",Cayuga County,13522,17384,33172,Cayuga County,181,81.0,100.0,0.447514,0.407633,166.938272,-47.679012
6094,New York,NY,"Chautauqua, NY",Chautauqua County,19091,31594,54234,Chautauqua County,847,605.0,242.0,0.714286,0.352012,31.555372,-20.666116
6104,New York,NY,"Chemung, NY",Chemung County,13757,20097,36119,Chemung County,160,46.0,114.0,0.287500,0.380880,299.065217,-137.826087
6114,New York,NY,"Chenango, NY",Chenango County,6775,11921,20160,Chenango County,5,2.0,3.0,0.400000,0.336062,3387.500000,-2573.000000
6124,New York,NY,"Clinton, NY",Clinton County,15059,14449,32105,Clinton County,403,143.0,260.0,0.354839,0.469055,105.307692,4.265734


In [273]:
df

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody


In [17]:
def add_county_18(df,state, county):
    from_df = statecountyimm[statecountyimm.State_init == state]
    from_df = from_df[from_df.County == str(county)]
    return from_df
add_county_18(df,'NC','Chatham County')

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
990,North Carolina,NC,"Chatham, NC",Chatham County,21065,17105,39849,Chatham County,132,55.0,77.0,0.416667,0.528621,383.0,72.0


In [18]:
list(statecountyimm.County[statecountyimm.State_init =='ND'])[:5]

[]

In [19]:
countylist = 'Androscoggin,Aroostook,Cumberland,Hancock,Kennebec,Knox,Lincoln,Oxford,Penobscot,Piscataquis,Somerset,Waldo,Washington,York'
countylist = 'Allegany,Broome,Cayuga,Chautauqua,Clinton,Delaware,Essex,Franklin,Jefferson,Montgomery,Oneida,Onondaga,Ontario,Orange,Oswego,Otsego,Tompkins,Ulster'
countylist = 'Benton,Coos,Crook,Gilliam,Jackson,Jefferson,Josephine,Linn,Multnomah,Polk,Union,Wallowa,Wasco,Yamhill'
countylist = 'Berkeley,Beaufort,Allendale,Chesterfield,Cherokee,Hampton,Kershaw'
for i in countylist.split(','):
    print "'"+i+" County" +"'"+',',

'Berkeley County', 'Beaufort County', 'Allendale County', 'Chesterfield County', 'Cherokee County', 'Hampton County', 'Kershaw County',


In [20]:
county_18_dict = {}

county_18_dict['AL'] = list(statecountyimm.County[statecountyimm.State_init =='AL'])
county_18_dict['AR'] = list(statecountyimm.County[statecountyimm.State_init =='AR'])
county_18_dict['CA'] = list(statecountyimm.County[statecountyimm.State_init =='CA'])
county_18_dict['CO'] = list(statecountyimm.County[statecountyimm.State_init =='CO']) # take out Denver and Bloomfield
county_18_dict['DE'] = list(statecountyimm.County[statecountyimm.State_init =='DE'])
county_18_dict['IL'] = list(statecountyimm.County[statecountyimm.State_init =='IL'])
county_18_dict['IN'] = list(statecountyimm.County[statecountyimm.State_init =='IN'])
county_18_dict['KY'] = list(statecountyimm.County[statecountyimm.State_init =='KY'])
county_18_dict['LA'] = ['Orleans County']
county_18_dict['ME'] = ['Androscoggin County', 'Aroostook County', 'Cumberland County', 'Hancock County', 'Kennebec County', 'Knox County', 'Lincoln County', 'Oxford County', 'Penobscot County', 'Piscataquis County', 'Somerset County', 'Waldo County', 'Washington County', 'York County']
county_18_dict['MD'] = list(statecountyimm.County[statecountyimm.State_init =='MD'])
county_18_dict['MN'] = list(statecountyimm.County[statecountyimm.State_init =='MN'])
county_18_dict['MT'] = list(statecountyimm.County[statecountyimm.State_init =='MT'])
county_18_dict['NE'] = list(statecountyimm.County[statecountyimm.State_init =='NE'])
county_18_dict['NV'] = list(statecountyimm.County[statecountyimm.State_init =='NV'])
county_18_dict['NJ'] = ['Camden County','Essex County','Gloucester County','Salem County']
county_18_dict['NY'] = ['Allegany County', 'Broome County', 'Cayuga County', 'Chautauqua County', 'Clinton County', 'Delaware County', 'Essex County', 'Franklin County', 'Jefferson County', 'Montgomery County', 'Oneida County', 'Onondaga County', 'Ontario County', 'Orange County', 'Oswego County', 'Otsego County', 'Tompkins County', 'Ulster County']
county_18_dict['NC'] = list(statecountyimm.County[statecountyimm.State_init =='NC'])
county_18_dict['ND'] = list(statecountyimm.County[statecountyimm.State_init =='ND'])
county_18_dict['OR'] = ['Benton County', 'Coos County', 'Crook County', 'Gilliam County', 'Jackson County', 'Jefferson County', 'Josephine County', 'Linn County', 'Multnomah County', 'Polk County', 'Union County', 'Wallowa County', 'Wasco County', 'Yamhill County']
county_18_dict['SC'] = ['Berkeley County', 'Beaufort County', 'Allendale County', 'Chesterfield County', 'Cherokee County', 'Hampton County', 'Kershaw County']
county_18_dict['SD'] = list(statecountyimm.County[statecountyimm.State_init =='SD'])
county_18_dict['TN'] = list(statecountyimm.County[statecountyimm.State_init =='TN'])
county_18_dict['UT'] = list(statecountyimm.County[statecountyimm.State_init =='UT'])
county_18_dict['VT'] = list(statecountyimm.County[statecountyimm.State_init =='VT'])
county_18_dict['WA'] = list(statecountyimm.County[statecountyimm.State_init =='WA']) # Take out Snohomish, Whatcom, & Pierce
county_18_dict['WI'] = list(statecountyimm.County[statecountyimm.State_init =='WI'])
county_18_dict['WY'] = list(statecountyimm.County[statecountyimm.State_init =='WY'])




In [ ]:
df = pd.DataFrame(columns = statecountyimm.columns)


In [278]:
df = pd.concat([df, add_county_18(df,"NC","Chatham County")])
df

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
5149,North Carolina,NC,"Chatham, NC",Chatham County,21065.0,17105.0,39849.0,Chatham County,132.0,55.0,77.0,0.416667,0.528621,383.0,72.0


In [21]:
df = pd.DataFrame(columns = statecountyimm.columns)
for k in county_18_dict:
    full_list = county_18_dict[k]
    for county in full_list:
        da = add_county_18(df,k, county)
        df = pd.concat([df,da])
df.head()
    

,State,State_init,countyname,fips_countyname_county,clinton,trump,total_votes,County,total_detained,ICE_custody,No_ICE_custody,pct_ice,pct_clinton,dem_votes_per_custody,dem_excess_per_custody
1881,Washington,WA,"Adams, WA",Adams County,1299.0,3083.0,4782.0,Adams County,232.0,175.0,57.0,0.754310,0.271644,7.422857,-10.194286
1882,Washington,WA,"Asotin, WA",Asotin County,3134.0,5741.0,9974.0,Asotin County,11.0,6.0,5.0,0.545455,0.314217,522.333333,-434.500000
1883,Washington,WA,"Benton, WA",Benton County,26360.0,47194.0,83592.0,Benton County,1695.0,1322.0,373.0,0.779941,0.315341,19.939486,-15.759455
1884,Washington,WA,"Chelan, WA",Chelan County,13032.0,18114.0,34433.0,Chelan County,1817.0,1507.0,310.0,0.829389,0.378474,8.647644,-3.372263
1885,Washington,WA,"Clallam, WA",Clallam County,17677.0,18794.0,40533.0,Clallam County,25.0,16.0,9.0,0.640000,0.436114,1104.812500,-69.812500


In [26]:
def graph_elex_18(detained_num=0, save=False):
    df = pd.DataFrame(columns = statecountyimm.columns)
    for k in county_18_dict:
        full_list = county_18_dict[k]
        for county in full_list:
            da = add_county_18(df,k, county)
            df = pd.concat([df,da])
    df = df[df.total_detained>detained_num]
    x = df.total_detained
    y = df.pct_ice
    colors = df.pct_clinton
    cm = plt.cm.get_cmap('RdBu')
    area= 400
    # area = (df.total_votes/df.total_detained)*10
    text = df['countyname']

    plt.rc('font', weight='bold', family='sans-serif', size=12)


    fig,ax = plt.subplots(figsize=(10,10))
    sct = ax.scatter(x, y, linewidths=2, s=area, edgecolor='w',c=colors,cmap=cm)
    # plugins.connect(fig, plugins.PointLabelTooltip(fig))
    sct.set_alpha(0.75)

    # tooltip = mpld3.plugins.PointLabelTooltip(sct, labels=zip(labels,label2))
    # mpld3.plugins.connect(fig, tooltip)

    
#     for labeli, xi, yi in zip(text, x, y):
#         ax.annotate(labeli,xy=(xi, yi))
            
            
    for labeli, xi, yi in zip(text, x, y):
        if xi>5000:
            ax.annotate(labeli,xy=(xi, yi))
#         if yi>(.9*max(y)):
#             ax.annotate(labeli,xy=(xi, yi))

    hfont = {'fontname':'DejaVu Sans'}

    
    ax.set_xlabel('Total Detained', fontsize=16,**hfont)
    ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
    ax.set_title('Counties Nationwide',fontsize=22,**hfont)

    # sct_html = fig_to_html(fig)
    if save==True:
        mpld3.save_html(fig,'../findings/oct_23_max/elections_18_counties.html')
    return mpld3.display()

graph_elex_18(detained_num=100, save=True)

In [32]:
sct

In [172]:
mpld3.save_html(fig,'test.html')

In [97]:
imm.head()

,FIPS,countyname,state,clinton,trump,johnson,ind_cand,mcmullin,total_votes,AGI,...,Statewide_UA_pop,County_y,total_detained,ICE_custody,No_ICE_custody,pct_ice,taxpayer_turnout,pct_clinton,pct_ua_pop_detained,log_total_detained
0,1073,"Jefferson, AL",AL,156873,134768,6645,12550,0,304191,18705559.0,...,71000,Jefferson County,508.0,41.0,467.0,0.080709,1.029202,0.515706,0.039077,6.230481
1,5007,"Benton, AR",AR,28005,60871,3871,7948,2313,96824,9646077.0,...,56000,Benton County,1196.0,764.0,432.0,0.638796,0.917589,0.289236,0.108727,7.086738
2,5143,"Washington, AR",AR,33366,41476,3360,7019,1438,81861,5088192.0,...,56000,Washington County,1058.0,556.0,502.0,0.525520,0.881268,0.407593,0.096182,6.964136
3,4027,"Yuma, AZ",AZ,24605,25165,1873,3240,0,53010,3258575.0,...,244000,Yuma County,479.0,241.0,238.0,0.503132,0.640449,0.464158,0.036846,6.171701
4,4021,"Pinal, AZ",AZ,47892,72819,5010,8835,0,129546,6370328.0,...,244000,Pinal County,3869.0,1622.0,2247.0,0.419230,0.970382,0.369691,0.351727,8.260751


In [89]:
# using imm, looking at pct ua pop detained
# imm includes an estimate of total number of unauthorized immigrants in a county
df = imm
x = df.pct_ua_pop_detained
y = df.pct_ice
colors = df.pct_clinton
cm = plt.cm.get_cmap('RdBu')
area = df.total_detained/10
text = df['countyname']

plt.rc('font', weight='bold', family='sans-serif', size=12)



fig,ax = plt.subplots(figsize=(10,10))
sct = ax.scatter(x, y, s=area, linewidths=2, edgecolor='w',c=colors, cmap=cm)
# plugins.connect(fig, plugins.PointLabelTooltip(fig))
sct.set_alpha(0.75)


df.total_detained = df['total_detained'].apply(lambda x: int(x) if x == x else "")

labels = ['{}'.format(i) for i in text]
label2 = ['{} detainees'.format(i) for i in df.total_detained]

tooltip = mpld3.plugins.PointLabelTooltip(sct, labels=zip(labels,label2))
mpld3.plugins.connect(fig, tooltip)

for labeli, xi, yi,si in zip(text, x, y,area):
    if si>1000:
        ax.annotate(labeli,xy=(xi, yi))

hfont = {'fontname':'DejaVu Sans'}

ax.set_xlabel('Percentage of Unauthorized Pop Detained', fontsize=16,**hfont)
ax.set_ylabel('Percentage of Detainees in ICE custody ',fontsize=16,**hfont)
ax.set_title('Frequency of Unauthorized Detainees in Custody',fontsize=22,**hfont)


# sct_html = fig_to_html(fig)
# mpld3.save_html(fig,'../findings/all_est_ua_counties.html')
mpld3.display()

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [47]:
import matplotlib.pyplot as plt, mpld3
%matplotlib inline
from mpld3 import fig_to_html,plugins
fig, ax = plt.subplots()
points = ax.plot(range(10), 'o')
lines = ax.plot(range(10), '-k')

mpld3.plugins.get_plugins(fig)
mpld3.plugins.BoxZoom(button=True)
mpld3.plugins.connect(fig,plugins.BoxZoom())
plugins.connect(fig,plugins.PointLabelTooltip(fig))
plugins.connect(fig, plugins.LineLabelTooltip(lines[0]))

mpld3.display()
# s = plt.plot([3,1,4,1,5], 'ks-', mec='w', mew=5, ms=20)

